# Classificando Talhões com Arquitetura CNN Personalizada

Este notebook descreve a implementação da nossa própria arquitetura de CNN. O objetivo do treinamento é classificar imagens e identificar se apresentam talhões ou não.

Com base nisso, utilizamos técnicas e arquiteturas já existentes como base para a implementação, descritas detalhadamente a seguir no notebook.

## Imports e setup da GPU

In [ ]:
import tensorflow as tf
tf.config.run_functions_eagerly(True)
from tensorflow import keras
import numpy as np
import cv2
import random
import os
import matplotlib.pyplot as plt
import time
from google.colab import drive

## Conexão com o Drive

In [ ]:
drive.mount('/content/drive')

## Conferência da existência de uma GPU

É preciso a GPU para fazer o teste de performance do código com GPU e com CPU

In [ ]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

# Processamento das imagens e separação entre dados de teste e treino

As imagens que serão processadas e consumidas pelo modelo foram préviamente processadas em nossa pipeline. Atualmente, no contexto da Sprint 2, a pipeline pode ser encontrada no link abaixo:

[Pré processamento das imagens para o modelo de classificação](../../SPRINT%202/MODELO%20CLASSIFICAÇÃO/20240509%20-%20Processamento%20de%20imagens%20para%20o%20modelo.ipynb)

Caso haja interesse em rodar e verificar a funcionalidade das funções, você pode encontrar um dataset que exemplifica essas imagens em amostra menor em: [Imagens com e sem talhão](../../../../data/SPRINT%202/MODELO%20DE%20CLASSIFICACAO/)

## Funções de vetorização das imagens e labels

A primeira função vetoriza as imagens com talhão e salva em uma lista, também cria uma lista com os labels da imagem, que no caso serão todos 1, porque todas as imagens tem talhão. A segunda função vetoriza as imagens de desmatamento (sem talhão) e cria a label 0 para todas essa imagens. A lista com imagens de talhão é cortada para ter no máximo 150 imagens, porque a base de desmatamento tem apenas 176 images, mesmo usando o data augmentation como proposto em **Lourenço, M.; Estima, D.; Oliveira, H.; Oliveira, L.; Mora, A. Automatic Rural Road Centerline Detection and Extraction from Aerial Images for a Forest Fire Decision Support System. Remote Sens. 2023, 15, 271.** https://doi.org/10.3390/rs15010271

In [ ]:
def vetorizar_imagem(caminho_desmatamento, label):
    lista_imagens = []
    labels = []
    for nome_imagem in os.listdir(caminho_desmatamento):
        caminho_imagem = os.path.join(caminho_desmatamento, nome_imagem)
        img = cv2.imread(caminho_imagem)
        if img is not None:  # Verifica se a imagem foi carregada corretamente
            img = cv2.resize(img, (200, 200))  # Redimensiona a imagem para ter certeza que todas têm o mesmo tamanho
            lista_imagens.append(img)
            labels.append(label)
        else:
            print(f"Erro ao carregar a imagem: {caminho_imagem}")  # Informa se houver erro ao carregar alguma imagem
    return np.array(lista_imagens), np.array(labels)

## Vetorização das imagens dos talhões

Chamamos a função que vetoriza as imagens dos talhões,passando o caminho para a pasta que contém essas imagens, e salvamos o retorno. Pode-se perceber que a saída são 4321 imagens de 200x200x3. Para equalizar com o conjunto de dados que não tem talhão, reduzimos para uma lista ter apenas 150 imagens.

In [ ]:
with tf.device('/gpu:0'):
    # Com talhão
    caminho_com_talhao = '/content/drive/MyDrive/imagens_com_talhao'
    imagens_com_talhao, labels_com_talhao = vetorizar_imagem(caminho_com_talhao, 1)
    print(imagens_com_talhao.shape)

In [ ]:
imagens_com_talhao = imagens_com_talhao[:150]
labels_com_talhao = labels_com_talhao[:150]

print(imagens_com_talhao.shape)
print(labels_com_talhao.shape)

## Vetorização das imagens de desmatamento


Chamamos a função que vetoriza as imagens de desmatamento, passando o caminho para a pasta que contém essas imagens, e salvamos o retorno. Pode-se perceber que a saída são 176 imagens de 200x200x3.

In [ ]:
with tf.device('/gpu:0'):
    caminho_desmatamento = '/content/drive/MyDrive/cropped_desmatamento'
    imagens_sem_talhao, labels_sem_talhao = vetorizar_imagem(caminho_desmatamento, 0)

    print(imagens_sem_talhao.shape)

In [ ]:
labels_sem_talhao.shape

## Combinação e normalização das imagens

Combinamos as imagens de talhão e desmatamento em uma só lista e também juntamos as labels. Após isso, normalizamos a lista das imagens.

In [ ]:
with tf.device('/gpu:0'):
    combined_images = np.concatenate((imagens_com_talhao, imagens_sem_talhao), axis=0)
    combined_labels = np.concatenate((labels_com_talhao, labels_sem_talhao), axis=0)

    # Normalização das imagens
    combined_images = combined_images / 255.0

    # Imprimir tamanhos dos arrays combinados para confirmar a correção
    print("Shape of the combined images array:", combined_images.shape)
    print("Shape of the combined labels array:", combined_labels.shape)


## Separação entre dados de teste e treino

Fazemos a separação entre dados de teste e de treino. Há também o embaralhamento da lista de imagens para que não fiquem apenas imagens de desmatamento no teste e vice-versa.

In [ ]:
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
(trainX, testX, trainY, testY) = train_test_split(combined_images, combined_labels)

# O código abaixo converterá os rótulos em vetores one-hot encoding
trainY = to_categorical(trainY, 2)
testY = to_categorical(testY, 2)
print(testY)

# Modelo: Implementação da LeNet

## Arquitetura do modelo

Nossa referência para a criação do modelo foram duas: LENET e outra dada em aula pelo Prof. Raphael. A primeira pode ser vista a semelhança nas duas primeiras camadas de convolução, com o número de neurônios da camada sendo igual a LENET. Diminuímos o tamanho do kernel, em relação a LENET, para poder ter maior precisão na detecção de bordas dos talhões. De acordo com o que vimos na aula do Prof. Raphael, aumentamos o número de camadas para poder extrair características mais complexas da imagem. Nosso modelo tem o mesmo número de camadas e pooling do notebook da aula, entretanto mudamos o número de neurônios e o tamanho da janela do pooling para diminuir os parâmetros treináveis.

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(6, (3, 3), activation='relu', input_shape=(200, 200, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(16, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(16, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(16, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(3,3),
    tf.keras.layers.Conv2D(16, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')
])

model.summary()


## Métricas, função de perda e otimizador do modelo

O modelo de classificação implementado utiliza a função de perda `binary_crossentropy`, que é ideal para problemas de classificação binária. Esta função é eficaz porque minimiza a distância entre as distribuições de probabilidade previstas e os rótulos verdadeiros, auxiliando o modelo a gerar previsões mais precisas, conforme destacado por Goodfellow et al. (2016).

Para a avaliação do desempenho do modelo, utilizamos `accuracy`, uma das mais intuitivas e comumente usadas em problemas de classificação. Ela quantifica a proporção de previsões corretas feitas pelo modelo, proporcionando uma medida direta de sua capacidade de classificar corretamente as imagens (Zhang et al., 2018).

O otimizador Adam é utilizado no modelo devido à sua eficácia em redes neurais profundas, adaptando automaticamente a taxa de aprendizagem e sendo computacionalmente eficiente. (Kingma e Ba, 2014).


**Referências**
- Goodfellow, I., Bengio, Y., & Courville, A. (2016). *Deep Learning*. MIT Press. [http://www.deeplearningbook.org](http://www.deeplearningbook.org)
- Kingma, D. P., & Ba, J. (2014). Adam: A method for stochastic optimization. [https://arxiv.org/abs/1412.6980](https://arxiv.org/abs/1412.6980)
- Zhang, Z., et al. (2018). *Understanding deep learning metrics and parameters.* [https://doi.org/10.1016/j.ymssp.2018.06.028](https://doi.org/10.1016/j.ymssp.2018.06.028)


In [ ]:
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Teste entre GPU e CPU

Comparar o desempenho de modelos treinados usando CPU e GPU é fundamental para entender como os recursos computacionais podem impactar a eficiência do treinamento. O processamento de dados em GPUs oferece paralelismo massivo e pode acelerar significativamente o treinamento de modelos de aprendizado profundo, especialmente com redes neurais complexas. No entanto, em alguns casos, as CPUs também podem ser eficientes, especialmente para modelos menores ou conjuntos de dados menos complexos.



## Função para o teste de processamento

Os tempos totais de treinamento e inferência são cronometrados e armazenados, permitindo análises comparativas entre as execuções em diferentes dispositivos. Esses tempos são visualizados através de gráficos de barras para facilitar a compreensão das diferenças de desempenho entre treinamento com CPU e GPU. Além disso, gráficos de linhas são utilizados para mostrar a evolução da acurácia e perda durante o treinamento, oferecendo insights sobre a eficácia do modelo ao longo das épocas em cada dispositivo. Esta abordagem não apenas destaca o impacto do hardware no desempenho do treinamento de modelos de aprendizado profundo, mas também ajuda a avaliar a eficiência do modelo em termos de velocidade e precisão.

In [ ]:
import time
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

def treinamento_e_inferência(trainX, testX, trainY, testY, device):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(6, (3, 3), activation='relu', input_shape=(200, 200, 3)),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(16, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(16, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(16, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(3, 3),
        tf.keras.layers.Conv2D(16, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(2, activation='softmax')
    ])

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    # Treinamento do modelo
    start_time = time.time()
    history = model.fit(trainX, trainY, batch_size=128, epochs=20, verbose=0,
                        validation_data=(testX, testY))
    training_time = time.time() - start_time

    # Inferência
    start_time = time.time()
    predictions = model.predict(testX, batch_size=64)
    inference_time = time.time() - start_time

    return training_time, inference_time, history

def plot_metrics(training_time, inference_time, history, device):
    # Plotando a acurácia e a perda
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Accuracy')
    plt.plot(history.history['val_accuracy'], label='Val Accuracy')
    plt.title(f'Accuracy over Epochs on {device}')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Loss')
    plt.plot(history.history['val_loss'], label='Val Loss')
    plt.title(f'Loss over Epochs on {device}')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

    # Plotando tempos de treinamento e inferência
    plt.figure(figsize=(7, 5))
    plt.bar(['Training', 'Inference'], [training_time, inference_time])
    plt.title(f'Training and Inference Time on {device}')
    plt.ylabel('Time (seconds)')
    plt.show()


## Teste entre GPU e CPU

Para testar o modelo na CPU, podemos alterar o ambiente de execução do Colab. No entanto, isso exige uma reinicialização completa do ambiente, o que pode ser inconveniente.

Através do código abaixo, podemos executar o modelo na CPU sem a necessidade de alterar o ambiente de execução.


#### Mensurando CPU

In [ ]:
# Treinando com CPU
with tf.device('/cpu:0'):
    print("CPU")
    training_time_cpu, inference_time_cpu, history_cpu = treinamento_e_inferência(trainX, testX, trainY, testY, 'CPU')
    plot_metrics(training_time_cpu, inference_time_cpu, history_cpu, 'CPU')


In [ ]:
print('Tempo de treino CPU: ', training_time_cpu)
print('Tempo de inferência CPU: ', inference_time_cpu)

#### Mensurando GPU

Através do código abaixo, podemos executar o modelo na GPU sem a necessidade de alterar o ambiente de execução.

In [ ]:
# Treinando com GPU
with tf.device('/gpu:0'):
    print("GPU")
    training_time_gpu, inference_time_gpu, history_gpu = treinamento_e_inferência(trainX, testX, trainY, testY, 'GPU')
    plot_metrics(training_time_gpu, inference_time_gpu, history_gpu, 'GPU')

In [ ]:
print('Tempo de treino GPU: ', training_time_gpu)
print('Tempo de inferência GPU: ', inference_time_gpu)

#### Analise dos resultados

Ao analisar os gráficos de Acurácia ao longo das épocas e a Perda, ambos os dispositivos mostram uma tendência consistente de melhoria na acurácia e redução da perda à medida que o número de épocas aumenta, indicando uma convergência eficaz do modelo em aprender a partir dos dados fornecidos.

O tempo de treinamento no GPU (10.22 segundos) é **drasticamente menor** em comparação com o CPU (202.24 segundos), ***refletindo a eficiência superior da GPU em manipular operações de matriz necessárias ***para o treinamento de redes neurais profundas.

 A GPU oferece vantagens claras para o treinamento de modelos, acelerando o processo e permitindo uma iteração mais rápida. Por outro lado, a decisão entre usar GPU ou CPU para inferência pode ser influenciada por considerações de custo e escalabilidade, dadas as semelhanças no tempo de execução.


# Treinamento e teste do modelo

In [ ]:
with tf.device('/gpu:0'):
    H = model.fit(trainX, trainY, batch_size=64, epochs=20, verbose=2,
            validation_data=(testX, testY))

In [ ]:
predictions = model.predict(testX, batch_size=16, verbose=2)

# Resultados

[inserir texto]


## Funções auxiliares para mensurar os resultados

In [ ]:
def plot_learning_curves(history):
    epochs = range(1, len(history.history['accuracy']) + 1)

    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.plot(epochs, history.history['accuracy'], label='Training Accuracy')
    plt.plot(epochs, history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Accuracy over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(epochs, history.history['loss'], label='Training Loss')
    plt.plot(epochs, history.history['val_loss'], label='Validation Loss')
    plt.title('Loss over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

def plot_confusion_matrix(testY, predictions):
    conf_mat = confusion_matrix(testY.argmax(axis=1), predictions.argmax(axis=1))
    sns.heatmap(conf_mat, annot=True, fmt='d', cmap='Blues', cbar=False,
                xticklabels=[str(label) for label in range(2)],
                yticklabels=[str(label) for label in range(2)])
    plt.xlabel('Predicted Labels')
    plt.ylabel('True Labels')
    plt.title('Confusion Matrix')
    plt.show()


In [ ]:
from sklearn.metrics import classification_report

def print_classification_report(testY, predictions):
    report = classification_report(testY.argmax(axis=1), predictions.argmax(axis=1),
                                   target_names=[str(label) for label in range(2)])
    print(report)

## Visualizando métricas


In [ ]:
plot_learning_curves(H)

In [ ]:
plot_confusion_matrix(testY, predictions)

In [ ]:
print_classification_report(testY, predictions)

Os resultados obtidos na avaliação do modelo de rede neural demonstram um desempenho promissor, evidenciado pela alta acurácia e pela redução consistente da perda ao longo das épocas de treinamento e validação.

A análise da perda durante as épocas mostra uma diminuição uniforme, apesar de pequenas oscilações nas últimas épocas, possivelmente devido à variação nos lotes de dados ou ajustes nos parâmetros. A tendência geral de queda na perda reitera a eficácia do processo de aprendizado.

As análises dispostas, complementadas pelos comentários durante a Sprint Review, mostram a necessidade de revisitarmos (se necessário for prosseguir com o modelo), a fim de que possamos testar e aprimorar a capacidade do modelo de generalizar seu conhecimento.

Contudo, uma análise mais profunda e comparativa pode ser encontrada na seção de Resultados do [artigo](../../artigo/artigo-grupo2.md).


Referências:

**How do you evaluate and compare different CNN models using metrics and plots?** *Disponível em:* <https://www.linkedin.com/advice/1/how-do-you-evaluate-compare-different-cnn-models#:~:text=For%20classification%20tasks%2C%20where%20you>. Acesso em: 11 maio. 2024.

‌